#### Workspaces

##### Data ingestion strategy:
<mark style="background: #88D5FF;">**REPLACE**</mark>

##### Related pipeline:

**Load_PBI_Workspaces_E2E**

##### Source:

**Files** from FUAM_Lakehouse folder **bronze_file_location** variable

##### Target:

**1 Delta table** in FUAM_Lakehouse 
- **gold_table_name** variable value


In [ ]:
import requests
from pyspark.sql.functions import col, lit, udf, explode, to_date, json_tuple, from_json, schema_of_json, get_json_object, upper
from pyspark.sql.types import StringType, json
from pyspark.sql import SparkSession
import json
from delta.tables import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
import datetime
spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled","true") # needed for automatic schema evolution in merge 

In [ ]:
## Parameters
display_data = True

#### Workspaces via PBI API

In [ ]:
## Variables
pbi_bronze_file_location = f"Files/raw/workspaces/"
pbi_silver_table_name = "FUAM_Staging_Lakehouse.workspaces_silver"
pbi_gold_table_name = "workspaces"
pbi_gold_table_name_with_prefix = f"Tables/{pbi_gold_table_name}"

In [ ]:
# Clean Silver table, if exists
if spark.catalog.tableExists(pbi_silver_table_name):
    del_query = "DELETE FROM " + pbi_silver_table_name
    spark.sql(del_query)

In [ ]:
# Get Bronze data
bronze_df = spark.read.option("multiline", "true").json(pbi_bronze_file_location)

In [ ]:
# Explode json subset structure
exploded_df = bronze_df.select(explode("value").alias("d"))

# Extract json objects to tabular form
extracted_df = exploded_df.select(col("d.*"))

# Convert key(s) to upper case
extracted_df = extracted_df.withColumn("id", f.upper(f.col("id")))
extracted_df = extracted_df.withColumn("capacityId", f.upper(f.col("capacityId")))

# Generate empty description column in case it is not available
if  not ("description" in extracted_df.columns):
    print("Create empty description column")
    extracted_df = extracted_df.withColumn("description", lit(""))

# Select all columns
silver_df = extracted_df.select(
    col("capacityId").alias("CapacityId"),
    col("id").alias("WorkspaceId"),
    col("capacityMigrationStatus").alias("CapacityMigrationStatus"),
    col("defaultDatasetStorageFormat").alias("DefaultDatasetStorageFormat"),
    col("description").alias("Description"),
    col("hasWorkspaceLevelSettings ").alias("HasWorkspaceLevelSettings"),
    col("isOnDedicatedCapacity").alias("IsOnDedicatedCapacity"),
    col("isReadOnly").alias("IsReadOnly"),
    col("name").alias("WorkspaceName"),
    col("state").alias("State"),
    col("type").alias("Type")
    )


In [ ]:
if display_data:
    display(silver_df)

In [ ]:
# Write prepared bronze_df to silver delta table
silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(pbi_silver_table_name)

In [ ]:

# This function maps and merges the silver data to gold dynamically
def write_silver_to_gold(pbi_silver_table_name, gold_table_name, ids):
    # TODO
    query = "SELECT *, current_timestamp() AS fuam_modified_at, False as fuam_deleted  FROM " + pbi_silver_table_name 
    silver_df = spark.sql(query)
    
    if spark._jsparkSession.catalog().tableExists('FUAM_Lakehouse', pbi_gold_table_name):
        # if exists -> MERGE to gold
        print("Gold table exists and will be merged.")
        gold_df = DeltaTable.forName(spark, pbi_gold_table_name)


        gold_columns = gold_df.toDF().columns
        silver_columns = silver_df.columns
        combined_columns = list(set(gold_columns) | set(silver_columns))
        id_cols = {}
        merge_id_stmt = ''
        for col in combined_columns:
            if col in ids:
                merge_id_stmt =  merge_id_stmt +  " t." + col + " = s." + col + " and"
                id_cols[col] = "s." + col

                
        # delete last and in merge id statement
        merge_id_stmt = merge_id_stmt[:-4]


        # Merge silver (s = source) to gold (t = target)
        try:
            merge = (gold_df.alias('t') \
            .merge(silver_df.alias('s'), merge_id_stmt )) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
            .whenNotMatchedBySourceUpdate( condition = "t.fuam_deleted == False or t.fuam_deleted IS NULL", set = {"fuam_deleted" : "True", "fuam_modified_at": "current_timestamp()"} )
            
            merge.execute()
        except:
        # In case the tables already exist, but the fuam column are not existent because of an old version do merge whenNotMatchedBySourceUpdate
            merge = (gold_df.alias('t') \
            .merge(silver_df.alias('s'), merge_id_stmt )) \
            .whenMatchedUpdateAll() \
            .whenNotMatchedInsertAll() \
                        
            merge.execute()

    else:
        # else -> INSERT to gold
        print("Gold table will be created.")

        silver_df.write.mode("append").option("mergeSchema", "true").format("delta").saveAsTable(pbi_gold_table_name)

In [ ]:
# Merge semantic model refreshes to gold table
write_silver_to_gold(pbi_silver_table_name, pbi_gold_table_name, ['WorkspaceId'])

In [ ]:
# write history of bronze files
mssparkutils.fs.cp(pbi_bronze_file_location, pbi_bronze_file_location.replace("Files/raw/", "Files/history/") + datetime.datetime.now().strftime('%Y/%m/%d') + "/", True)

#### Workspaces via Fabric API

In [ ]:
## Variables
fb_bronze_file_location = f"Files/raw/fabric_workspaces/"
fb_silver_table_name = "FUAM_Staging_Lakehouse.fabric_workspaces_silver"
fb_gold_table_name = "fabric_workspaces"
fb_gold_table_name_with_prefix = f"Tables/{fb_gold_table_name}"

In [ ]:
# Clean Silver table, if exists
if spark.catalog.tableExists(fb_silver_table_name):
    del_query = "DELETE FROM " + fb_silver_table_name
    spark.sql(del_query)

In [ ]:
# Get Bronze data
fb_bronze_df = spark.read.option("multiline", "true").json(fb_bronze_file_location)

In [ ]:
if display_data:
    display(fb_bronze_df)

In [ ]:
# Explode json subset structure
fb_exploded_df = fb_bronze_df.select(explode("workspaces").alias("d"))
# Extract json objects to tabular form
fb_extracted_df = fb_exploded_df.select(col("d.*"))


if "domainId" not in fb_extracted_df.columns:
    fb_extracted_df = fb_extracted_df.withColumn("domainId",lit(None) )

fb_extracted_df = fb_extracted_df.withColumnRenamed("capacityId", "CapacityId")\
.withColumnRenamed("id", "WorkspaceId")\
.withColumnRenamed("domainId", "DomainId")

fb_extracted_df = fb_extracted_df.withColumn("DomainId", upper("DomainId"))\
.withColumn("CapacityId", upper("CapacityId"))\
.withColumn("WorkspaceId", upper("WorkspaceId"))\


if display_data:
    display(fb_extracted_df)

In [ ]:
# Rename columns if needed
fb_silver_df = fb_extracted_df

if display_data:
    display(fb_silver_df)

In [ ]:
# Write prepared bronze_df to silver delta table
fb_silver_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable(fb_silver_table_name)

In [ ]:
# Get Silver table data
query = """
SELECT *
FROM """ + fb_silver_table_name


fb_silver_df = spark.sql(query)

if display_data:
     display(fb_silver_df)

In [ ]:
#fb_silver_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable(fb_gold_table_name)

#### Merge Workspace Tables

In [ ]:
df_workspaces = spark.sql("SELECT * FROM FUAM_Lakehouse.workspaces")
display(df_workspaces)

if "DomainId" not in df_workspaces.columns:
    spark.sql("ALTER TABLE workspaces ADD COLUMNS (DomainId STRING) ")

In [ ]:
%%sql
MERGE INTO workspaces trg
USING FUAM_Staging_Lakehouse.fabric_workspaces_silver src 
ON trg.WorkspaceId = src.WorkspaceId
WHEN MATCHED THEN
UPDATE SET DomainId = src.DomainId

In [ ]:
# Write history of bronze files
from datetime import datetime
mssparkutils.fs.cp(fb_bronze_file_location, fb_bronze_file_location.replace("Files/raw/", "Files/history/") + datetime.now().strftime('%Y/%m/%d') + "/", True)